In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 391016
drwxr-xr-x 1 root root      4096 Jan 11 17:02 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

In [ ]:
ls

sample_data/  spark-3.5.0-bin-hadoop3/  spark-3.5.0-bin-hadoop3.tgz


In [ ]:
CarsDF = spark.read.option("inferSchema", True).option("header", True).csv("./RoadAccidentData.csv")
CarsDF.show()

+--------------+-------------+-----------+--------------------+--------------------+-----------------+---------+--------------------+--------------------------+-------------------+---------+--------------------+------------------+-------------------+-----------------------+------------------+-----------+-------------------+-------------------+--------------------+--------------------+
|Accident_Index|Accident Date|Day_of_Week|    Junction_Control|     Junction_Detail|Accident_Severity| Latitude|    Light_Conditions|Local_Authority_(District)|Carriageway_Hazards|Longitude|Number_of_Casualties|Number_of_Vehicles|       Police_Force|Road_Surface_Conditions|         Road_Type|Speed_limit|               Time|Urban_or_Rural_Area|  Weather_Conditions|        Vehicle_Type|
+--------------+-------------+-----------+--------------------+--------------------+-----------------+---------+--------------------+--------------------------+-------------------+---------+--------------------+-------------

In [ ]:
CarsDF.printSchema()

root
 |-- Accident_Index: string (nullable = true)
 |-- Accident Date: string (nullable = true)
 |-- Day_of_Week: string (nullable = true)
 |-- Junction_Control: string (nullable = true)
 |-- Junction_Detail: string (nullable = true)
 |-- Accident_Severity: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Light_Conditions: string (nullable = true)
 |-- Local_Authority_(District): string (nullable = true)
 |-- Carriageway_Hazards: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Number_of_Casualties: integer (nullable = true)
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Police_Force: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- Weather_Conditions: string (nullable = true)
 |-- Vehicle_Type: string (nullable = true)



In [ ]:
CarsDF.show(2)

+--------------+-------------+-----------+--------------------+--------------------+-----------------+---------+----------------+--------------------------+-------------------+---------+--------------------+------------------+-------------------+-----------------------+------------------+-----------+-------------------+-------------------+------------------+--------------------+
|Accident_Index|Accident Date|Day_of_Week|    Junction_Control|     Junction_Detail|Accident_Severity| Latitude|Light_Conditions|Local_Authority_(District)|Carriageway_Hazards|Longitude|Number_of_Casualties|Number_of_Vehicles|       Police_Force|Road_Surface_Conditions|         Road_Type|Speed_limit|               Time|Urban_or_Rural_Area|Weather_Conditions|        Vehicle_Type|
+--------------+-------------+-----------+--------------------+--------------------+-----------------+---------+----------------+--------------------------+-------------------+---------+--------------------+------------------+----------

In [ ]:
MoredayaccidentDF = CarsDF.groupBy("Light_Conditions").agg(count("Accident_Index").alias("Accident_Count")).show()


+--------------------+--------------+
|    Light_Conditions|Accident_Count|
+--------------------+--------------+
|Darkness - lights...|            49|
|Darkness - lighti...|            70|
|            Daylight|         19842|
|Darkness - lights...|          7401|
|Darkness - no lig...|           306|
|                NULL|             1|
+--------------------+--------------+



Accidente con mas coches implicados

In [ ]:
MorecarsDF = CarsDF.groupBy("Accident_Index").agg(max("Number_of_Vehicles").alias("Max_Number_of_Vehicles"))
MaxCarsAccident = MorecarsDF.orderBy("Max_Number_of_Vehicles", ascending=False).limit(1)
MaxCarsAccident.show()

+--------------+----------------------+
|Accident_Index|Max_Number_of_Vehicles|
+--------------+----------------------+
| 200901BS70611|                     9|
+--------------+----------------------+



In [ ]:
resultDF = carsDF.groupBy("Day_of_Week").count()
resultDF = resultDF.orderBy(col("count").desc())
resultDF = resultDF.withColumnRenamed("count", "Numero_de_Accidentes")
resultDF.show()

NameError: name 'carsDF' is not defined

In [ ]:
result2DF = CarsDF.groupBy("Junction_Control").count()
result2DF = result2DF.orderBy(col("count").desc())
result2DF = result2DF.withColumnRenamed("Junction_Control", "Lugar_del_Accidente")
result2DF = result2DF.withColumnRenamed("count", "Numero_de_Veces")
result2DF.show()

+--------------------+---------------+
| Lugar_del_Accidente|Numero_de_Veces|
+--------------------+---------------+
|Give way or uncon...|          14647|
|Data missing or o...|           7865|
| Auto traffic signal|           5068|
|           Stop sign|             70|
|   Authorised person|             19|
+--------------------+---------------+



In [ ]:
result3DF = CarsDF.groupBy("Day_of_Week").pivot("Accident_Severity").count()
result3DF = result3DF.withColumn("Total_Severity", expr("coalesce(Fatal, 0) + coalesce(Serious, 0) + coalesce(Slight, 0) + coalesce(Fetal, 0)"))
result3DF = result3DF.drop("Total_Severity")
result3DF = result3DF.select(col("Day_of_Week"), col("Fatal").alias("Fatal"), col("Fetal").alias("Fatal"), col("Serious").alias("Serio"), col("Slight").alias("Leve"))
result3DF = result3DF.orderBy(col("Fatal").desc())
result3DF.show()

+-----------+-----+-----+-----+----+
|Day_of_Week|Fatal|Fatal|Serio|Leve|
+-----------+-----+-----+-----+----+
|   Saturday|   89|    6|  490|3050|
|     Friday|   87|    7|  556|3939|
|     Monday|   69|    6|  453|3300|
|  Wednesday|   67|    7|  520|3640|
|   Thursday|   67|    9|  502|3581|
|    Tuesday|   65|   12|  505|3624|
|     Sunday|   56|    2|  438|2522|
+-----------+-----+-----+-----+----+



In [ ]:
result3DF = CarsDF.groupBy("Day_of_Week").pivot("Accident_Severity").count()
result3DF = result3DF.withColumn("Total_Severity", expr("coalesce(Fatal, 0) + coalesce(Serious, 0) + coalesce(Slight, 0) + coalesce(Fetal, 0)"))
result3DF = result3DF.withColumn("Porcentaje_Mortalidad", (col("Fatal") / col("Total_Severity")) * 100)
result3DF = result3DF.orderBy(col("Porcentaje_Mortalidad").desc()).select("Day_of_Week", "Porcentaje_Mortalidad")
result3DF.show()

+-----------+---------------------+
|Day_of_Week|Porcentaje_Mortalidad|
+-----------+---------------------+
|   Saturday|    2.448418156808803|
|     Friday|   1.8958378731749836|
|     Sunday|   1.8555334658714382|
|     Monday|   1.8025078369905956|
|   Thursday|   1.6109641740803076|
|  Wednesday|    1.582427964100142|
|    Tuesday|   1.5454113171659534|
+-----------+---------------------+

